In [49]:
import pandas as pd
import numpy as np
import requests

In [50]:
# Get general data
url_general_data = "https://fantasy.premierleague.com/api/bootstrap-static/"
r = requests.get(url_general_data)
general_data = r.json()

# Get fixture data for future fixture difficulty
url_fdr_data = "https://fantasy.premierleague.com/api/fixtures/?future=1"
response = requests.get(url_fdr_data)
fixtures_data = response.json()

# Get fixture data for all fixture difficulty
url_all_fdr_data = "https://fantasy.premierleague.com/api/fixtures/"
all_response = requests.get(url_all_fdr_data)
all_fixtures_data = all_response.json()


In [51]:
elements_df = pd.DataFrame(general_data['elements'])
reduced_elements_df = elements_df[['id', 'first_name', 'second_name', 'team', 'element_type', 'now_cost', 'selected_by_percent', 'points_per_game', 'ict_index']]
reduced_elements_df['element_type'] = reduced_elements_df['element_type'].map({1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD', 5: 'MGR'})
reduced_elements_df['now_cost'] = reduced_elements_df['now_cost']/10

C:\Users\arron\AppData\Local\Temp\ipykernel_31284\676363444.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_elements_df['element_type'] = reduced_elements_df['element_type'].map({1: 'GK', 2: 'DEF', 3: 'MID', 4: 'FWD', 5: 'MGR'})
C:\Users\arron\AppData\Local\Temp\ipykernel_31284\676363444.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_elements_df['now_cost'] = reduced_elements_df['now_cost']/10


In [52]:
teams_df = pd.DataFrame(general_data['teams'])

# merge the reduced_elements_df with the teams_df
reduced_elements_df = reduced_elements_df.merge(
    teams_df[['id', 'name']],  # Extract only the 'id' and 'name' columns from teams_df
    left_on='team',           # Match the 'team' column in reduced_elements_df
    right_on='id',            # Match the 'id' column in teams_df
    how='left'                # Perform a left join
)

# Similar to SQL joins

In [53]:
#drop the 'team' and 'id' columns, combine name
reduced_elements_df = reduced_elements_df.drop(columns=['team', 'id_y'])
reduced_elements_df['full_name'] = reduced_elements_df['first_name'] + " " + reduced_elements_df['second_name']
reduced_elements_df.drop(['first_name', 'second_name'], axis=1, inplace=True)

# rearrange columns
reduced_elements_df = reduced_elements_df[['id_x', 'full_name', 'name', 'element_type', 'now_cost', 'selected_by_percent', 'points_per_game', 'ict_index']]

# rename columns
reduced_elements_df.columns = ['player_id', 'full_name', 'team', 'position', 'price', 'selected_by_percent', 'points_per_game', 'ict_index']

# Convert columns to appropriate data types
reduced_elements_df['selected_by_percent'] = reduced_elements_df['selected_by_percent'].astype(float)
reduced_elements_df['points_per_game'] = reduced_elements_df['points_per_game'].astype(float)
reduced_elements_df['ict_index'] = reduced_elements_df['ict_index'].astype(float)

In [54]:
# data for future fixture difficulty
fixtures_df = pd.DataFrame(all_fixtures_data)
fdr_df = fixtures_df[['id', 'team_h', 'team_a', 'team_h_difficulty', 'team_a_difficulty']]

# data for all fixture difficulty
all_fixtures_df = pd.DataFrame(all_fixtures_data)
all_fdr_df = all_fixtures_df[['id', 'team_h', 'team_a', 'team_h_difficulty', 'team_a_difficulty']]

In [55]:
# future fixture difficulty

# merge home team
fdr_df = fdr_df.merge(
    teams_df[['id', 'name']],
    left_on='team_h', 
    right_on='id',
    how='left')

# merge away team
fdr_df = fdr_df.merge(
    teams_df[['id', 'name']],
    left_on='team_a', 
    right_on='id',
    how='left')

# all fixture difficulty

# merge home team
all_fdr_df = all_fdr_df.merge(
    teams_df[['id', 'name']],
    left_on='team_h', 
    right_on='id',
    how='left')

# merge away team
all_fdr_df = all_fdr_df.merge(
    teams_df[['id', 'name']],
    left_on='team_a', 
    right_on='id',
    how='left')

In [56]:
# clean fdr_df
fdr_df = fdr_df.drop(columns=['id_x', 'id_y'])
fdr_df = fdr_df.rename(columns={'name_x': 'home_team', 'name_y': 'away_team'})

# clean fdr_df
all_fdr_df = all_fdr_df.drop(columns=['id_x', 'id_y'])
all_fdr_df = all_fdr_df.rename(columns={'name_x': 'home_team', 'name_y': 'away_team'})

In [57]:
# FUTURE FIXTURE DIFFICULTY
# create seperate df for home and away teams
home_df = fdr_df[['id', 'home_team', 'team_h_difficulty']].rename(columns={'id': 'fixture_id', 'home_team': 'team', 'team_h_difficulty': 'difficulty'})
away_df = fdr_df[['id', 'away_team', 'team_a_difficulty']].rename(columns={'id': 'fixture_id', 'away_team': 'team', 'team_a_difficulty': 'difficulty'})

# combine home and away df
all_teams_df = pd.concat([home_df, away_df], ignore_index=True)

all_teams_df['fixture'] = all_teams_df.groupby('team').cumcount() + 1

# pivot so fixtures are columns
new_fdr_df = all_teams_df.pivot(index='team', columns='fixture', values='difficulty').reset_index()

# Rename the columns
new_fdr_df.columns.name = None
new_fdr_df = new_fdr_df.rename(columns=lambda x: f"Fixture {x}" if isinstance(x, int) else x)

# pivot so team names are columns
fdr_df1 = all_teams_df.pivot(index='fixture', columns='team', values='difficulty').reset_index()

# ALL FIXTURE DIFFICULTY
# create seperate df for home and away teams
all_home_df = all_fdr_df[['id', 'home_team', 'team_h_difficulty']].rename(columns={'id': 'fixture_id', 'home_team': 'team', 'team_h_difficulty': 'difficulty'})
all_away_df = all_fdr_df[['id', 'away_team', 'team_a_difficulty']].rename(columns={'id': 'fixture_id', 'away_team': 'team', 'team_a_difficulty': 'difficulty'})

# combine home and away df
all_all_teams_df = pd.concat([all_home_df, all_away_df], ignore_index=True)

all_all_teams_df['fixture'] = all_all_teams_df.groupby('team').cumcount() + 1

# pivot so fixtures are columns
all_new_fdr_df = all_all_teams_df.pivot(index='team', columns='fixture', values='difficulty').reset_index()

# Rename the columns
all_new_fdr_df.columns.name = None
all_new_fdr_df = all_new_fdr_df.rename(columns=lambda x: f"Fixture {x}" if isinstance(x, int) else x)

# pivot so team names are columns
all_fdr_df1 = all_all_teams_df.pivot(index='fixture', columns='team', values='difficulty').reset_index()


In [58]:
# average fixture rating
new_fdr_df['Next 6 Average FDR'] = new_fdr_df.iloc[:, 1:7].mean(axis=1)

reduced_elements_df = reduced_elements_df.merge(
    new_fdr_df[['team', 'Next 6 Average FDR']],
    on= 'team',
    how='left'              
)

In [59]:
# Establish ranges for the average FDR
ranges = {
    'GW 1-6': ['Fixture 1', 'Fixture 2', 'Fixture 3', 'Fixture 4', 'Fixture 5', 'Fixture 6'],  
    'GW 7-12': ['Fixture 7', 'Fixture 8', 'Fixture 9', 'Fixture 10', 'Fixture 11', 'Fixture 12'],
    'GW 13-18': ['Fixture 13', 'Fixture 14', 'Fixture 15', 'Fixture 16', 'Fixture 17', 'Fixture 18']  
}

for range_name, columns in ranges.items():
    all_new_fdr_df[f"Average FDR {range_name}"] = all_new_fdr_df[columns].mean(axis=1)

In [60]:
# Merge Average FDR for each range with the reduced_elements_df
reduced_elements_df = reduced_elements_df.merge(
    all_new_fdr_df[['team', 'Average FDR GW 1-6', 'Average FDR GW 7-12', 'Average FDR GW 13-18']],
    on= 'team',
    how='left'              
)

In [61]:
def fetch_player_history(player_id):
    """
    Fetch the history for a given player_id from the FPL API.
    Returns a DataFrame of the player's history (or an empty DataFrame if failed).
    """
    url = f"https://fantasy.premierleague.com/api/element-summary/{player_id}/"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return pd.DataFrame(data.get('history', []))
    else:
        print(f"Failed to retrieve data for player ID {player_id}")
        return pd.DataFrame()

In [62]:
# Getting form, xG, and xA data for each player

forms = []
xGs_last_6 = []
xAs_last_6 = []
xGs_season = []
xAs_season = []

for _, row in reduced_elements_df.iterrows():
    player_id = row['player_id']
    history = fetch_player_history(player_id)
    
    if not history.empty:
        # Ensure expected_goals and expected_assists are numeric
        history['expected_goals'] = pd.to_numeric(history['expected_goals'], errors='coerce')
        history['expected_assists'] = pd.to_numeric(history['expected_assists'], errors='coerce')
        
        # Extract the last 6 fixtures and calculate metrics
        last_6 = history.tail(6)
        form = last_6['total_points'].mean()
        xG_last = last_6['expected_goals'].sum()
        xA_last = last_6['expected_assists'].sum()
        
        # Season-long metrics: sum over all history
        xG_total = history['expected_goals'].sum()
        xA_total = history['expected_assists'].sum()
    else:
        form, xG_last, xA_last, xG_total, xA_total = 0, 0, 0, 0, 0

    forms.append(form)
    xGs_last_6.append(xG_last)
    xAs_last_6.append(xA_last)
    xGs_season.append(xG_total)
    xAs_season.append(xA_total)

reduced_elements_df['form'] = forms
reduced_elements_df['xG_last_6'] = xGs_last_6
reduced_elements_df['xA_last_6'] = xAs_last_6
reduced_elements_df['xG_season'] = xGs_season
reduced_elements_df['xA_season'] = xAs_season


In [63]:
# Rename the column
reduced_elements_df['last 6 form'] = forms

In [64]:
# Calculate points for each range

points_gw_1_6 = []
points_gw_7_12 = []
points_gw_13_18 = []

for _, row in reduced_elements_df.iterrows():
    player_id = row['player_id']
    history = fetch_player_history(player_id)
    
    if not history.empty:
        gw1_6 = history[history['round'].between(1, 6)]['total_points'].sum()
        gw7_12 = history[history['round'].between(7, 12)]['total_points'].sum()
        gw13_18 = history[history['round'].between(13, 18)]['total_points'].sum()
    else:
        gw1_6, gw7_12, gw13_18 = 0, 0, 0

    points_gw_1_6.append(gw1_6)
    points_gw_7_12.append(gw7_12)
    points_gw_13_18.append(gw13_18)

reduced_elements_df['points_gw_1_6'] = points_gw_1_6
reduced_elements_df['points_gw_7_12'] = points_gw_7_12
reduced_elements_df['points_gw_13_18'] = points_gw_13_18

In [65]:
# Getting minutes played for each player

minutes_played_list = []

for _, row in reduced_elements_df.iterrows():
    player_id = row['player_id']
    history = fetch_player_history(player_id)
    
    if not history.empty:
        gw1_6 = history[(history['round'] >= 1) & (history['round'] <= 6)]
        gw7_12 = history[(history['round'] >= 7) & (history['round'] <= 12)]
        gw13_18 = history[(history['round'] >= 13) & (history['round'] <= 18)]
        
        minutes_1_6 = gw1_6['minutes'].sum()
        minutes_7_12 = gw7_12['minutes'].sum()
        minutes_13_18 = gw13_18['minutes'].sum()
        total_minutes = history['minutes'].sum()
    else:
        minutes_1_6 = minutes_7_12 = minutes_13_18 = total_minutes = 0

    minutes_played_list.append({
        'player_id': player_id,
        'minutes_1_6': minutes_1_6,
        'minutes_7_12': minutes_7_12,
        'minutes_13_18': minutes_13_18,
        'total_minutes': total_minutes
    })

minutes_df = pd.DataFrame(minutes_played_list)
reduced_elements_df = reduced_elements_df.merge(minutes_df, on='player_id', how='left')


In [66]:
# Calculate the total points for each player in each gameweek

player_points_list = []

for _, row in reduced_elements_df.iterrows():
    player_id = row['player_id']
    player_name = row['full_name']
    history = fetch_player_history(player_id)
    
    if not history.empty:
        player_gw_points = history[['round', 'total_points']].copy()
        player_gw_points['player_id'] = player_id
        player_gw_points['player_name'] = player_name
        player_points_list.append(player_gw_points)
    else:
        # If no history exists, append an empty DataFrame for this player
        player_points_list.append(pd.DataFrame({
            'round': [], 
            'total_points': [], 
            'player_id': [player_id], 
            'player_name': [player_name]
        }))

# Combine into one DataFrame
player_points_df = pd.concat(player_points_list, ignore_index=True)

# Map team from reduced_elements_df to player_points_df
player_team_mapping = reduced_elements_df.set_index('player_id')['team'].to_dict()
player_points_df['team'] = player_points_df['player_id'].map(player_team_mapping)

# Rename columns in all_teams_df for clarity and merge FDR info
all_teams_df = all_teams_df.rename(columns={'fixture': 'round', 'difficulty': 'fdr'})
player_points_df = player_points_df.merge(
    all_teams_df[['team', 'round', 'fdr']],
    on=['team', 'round'],
    how='left'
)




In [67]:
# Filter gw6_df to only have data for gameweeks 1-6
gw6_df = player_points_df[player_points_df['round'] <= 6]

# Group by player_id (or player_id + team) and calculate average points
form_gw6_df = gw6_df.groupby('player_id')['total_points'].mean().reset_index()

# Rename the points column to something descriptive
form_gw6_df.rename(columns={'total_points': 'form_end_gw6'}, inplace=True)

# Merge the form_gw6_df with the reduced_elements_df

reduced_elements_df = reduced_elements_df.merge(
    form_gw6_df,
    on='player_id',
    how='left'
)


In [68]:
# Calculate total points scored by each player
overall_points_df = player_points_df.groupby(['player_id'])['total_points'].sum().reset_index()

# Rename column for clarity
overall_points_df.rename(columns={'total_points': 'overall_points'}, inplace=True)

# Merge with reduced_elements_df to add overall_points
reduced_elements_df = reduced_elements_df.merge(
    overall_points_df, 
    on='player_id', 
    how='left'
)

# Display the updated DataFrame
print(reduced_elements_df.head())


   player_id                     full_name     team position  price  \
0          1         Fábio Ferreira Vieira  Arsenal      MID    5.4   
1          2     Gabriel Fernando de Jesus  Arsenal      FWD    6.6   
2          3  Gabriel dos Santos Magalhães  Arsenal      DEF    6.3   
3          4                   Kai Havertz  Arsenal      FWD    7.8   
4          5                     Karl Hein  Arsenal       GK    4.0   

   selected_by_percent  points_per_game  ict_index  Next 6 Average FDR  \
0                  0.0              0.0        0.0            2.833333   
1                  1.3              2.5       52.6            2.833333   
2                 27.5              4.4       91.7            2.833333   
3                  7.5              4.2      132.1            2.833333   
4                  0.0              0.0        0.0            2.833333   

   Average FDR GW 1-6  ...  last 6 form  points_gw_1_6  points_gw_7_12  \
0            2.833333  ...     0.000000              0

In [69]:
# Save the data to an Excel file

reduced_elements_df.to_excel('fpl_data.xlsx', index=False)
player_points_df.to_excel('player_points_data.xlsx', index=False)


In [70]:
reduced_elements_df.head()

,player_id,full_name,team,position,price,selected_by_percent,points_per_game,ict_index,Next 6 Average FDR,Average FDR GW 1-6,...,last 6 form,points_gw_1_6,points_gw_7_12,points_gw_13_18,minutes_1_6,minutes_7_12,minutes_13_18,total_minutes,form_end_gw6,overall_points
0,1,Fábio Ferreira Vieira,Arsenal,MID,5.4,0.0,0.0,0.0,2.833333,2.833333,...,0.000000,0,0,0,0,0,0,0,0.000000,0
1,2,Gabriel Fernando de Jesus,Arsenal,FWD,6.6,1.3,2.5,52.6,2.833333,2.833333,...,2.333333,2,5,21,19,192,209,600,0.333333,42
2,3,Gabriel dos Santos Magalhães,Arsenal,DEF,6.3,27.5,4.4,91.7,2.833333,2.833333,...,2.166667,37,14,28,540,503,315,1808,6.166667,92
3,4,Kai Havertz,Arsenal,FWD,7.8,7.5,4.2,132.1,2.833333,2.833333,...,1.666667,32,14,28,540,450,490,1750,5.333333,84
4,5,Karl Hein,Arsenal,GK,4.0,0.0,0.0,0.0,2.833333,2.833333,...,0.000000,0,0,0,0,0,0,0,0.000000,0


In [71]:
player_points_df.head()

,round,total_points,player_id,player_name,team,fdr
0,1,0,1,Fábio Ferreira Vieira,Arsenal,2
1,2,0,1,Fábio Ferreira Vieira,Arsenal,3
2,3,0,1,Fábio Ferreira Vieira,Arsenal,2
3,4,0,1,Fábio Ferreira Vieira,Arsenal,2
4,5,0,1,Fábio Ferreira Vieira,Arsenal,5
